# Entrenamiento Modelo
Una ves realizado la limpieza y definición de los datos, se proceden a cargar todos estos datos, al modelo para ser entrenado, en este caso se utiliza un modelo, para los datos que no cuentan con las reglas de lipinski 


## Requisitos previos:
Asegúrate de tener un entorno de Python con las siguientes bibliotecas instaladas: matplotlib, pandas, seaborn, scikit-learn, imbalanced-learn, optuna.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pickle
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import optuna
import os

Seleccion carpeta o localizacion de los datos

In [ ]:
print(os.getcwd())

os.chdir('/home/aamaya/BACE1')

### Paso 1: Cargar los datos de entrenamiento
El código carga los datos de entrenamiento previamente procesados desde el archivo 'datosentrenamientoSIN_df.pickle'.

In [ ]:
with open('datosentrenamientoSIN_df.pickle', 'rb') as f:
    df = pickle.load(f)

### Paso 2: Dividir los datos en entrenamiento y validación
Divide los datos en conjuntos de entrenamiento y validación. En este caso, se usa una proporción del 80% para entrenamiento y 20% para validación. Los datos se dividen de forma estratificada para mantener el equilibrio entre las clases.


In [ ]:
df_target, validacion = train_test_split(df, test_size=0.20, shuffle=True, stratify=df['active'])

### Paso 3: Guardar los datos de entrenamiento y validación en archivos pickle
Los conjuntos de datos de entrenamiento y validación se guardan en archivos pickle para su uso posterior.


In [ ]:
with open('df_targetSIN.pickle', 'wb') as f:
    pickle.dump(df_target, f)

### Paso 4: Cargar los datos de entrenamiento y validación desde archivos pickle
El código carga los conjuntos de datos de entrenamiento y validación desde los archivos pickle guardados.


In [ ]:
with open('df_targetSIN.pickle', 'rb') as f:
    df_train = pickle.load(f)
    
with open('validacionSIN.pickle', 'wb') as f:
    pickle.dump(validacion, f)

with open('validacionSIN.pickle', 'rb') as f:
    df_val = pickle.load(f)

X_train = df_train.np_bv.tolist()
y_train = df_train.active
X_val = df_val.np_bv.tolist()
y_val = df_val.active

### Paso 5: Aplicar SMOTE para el balanceo de clases
Se utiliza SMOTE (Técnica de Sobremuestreo de Minorías Sintéticas) para equilibrar las clases en los datos de entrenamiento.



In [ ]:
smote = SMOTE(sampling_strategy=0.7, random_state=200)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

### Paso 6: Definir métricas de evaluación
Se definen las métricas de evaluación que se utilizarán para evaluar el modelo, como precisión, recuperación, F1-score, área bajo la curva ROC (ROC-AUC) y coeficiente de correlación de Matthews (MCC).


In [ ]:
# Definir metricas de evaluacion
scoring = {
    'Accuracy': 'accuracy',
    'Precision': 'precision',
    'Recall': 'recall',
    'F1': 'f1',
    'ROC-AUC': 'roc_auc',
    'MCC': 'matthews_corrcoef'
}

### Paso 7: Definir la función de optimización de hiperparámetros
Se define la función objective que utiliza Optuna para optimizar los hiperparámetros del modelo Random Forest. Optuna busca la combinación de hiperparámetros que maximiza una métrica personalizada que combina las métricas de evaluación.



In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 10000, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 30, log=True),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20, log=True),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 8),
        'max_features': trial.suggest_categorical('max_features', [0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 'sqrt']),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced', 'balanced_subsample'])
    }
    rf = RandomForestClassifier(random_state=200, bootstrap=True, **params)
    rf_cv = cross_validate(rf, X_resampled, y_resampled, cv=StratifiedKFold(n_splits=5), scoring=scoring, error_score='raise', n_jobs=-1)
    accuracy_mean = rf_cv['test_Accuracy'].mean()
    precision_mean = rf_cv['test_Precision'].mean()
    recall_mean = rf_cv['test_Recall'].mean()
    f1_mean = rf_cv['test_F1'].mean()
    roc_auc_mean = rf_cv['test_ROC-AUC'].mean()
    mcc_mean = rf_cv['test_MCC'].mean()
    return -(accuracy_mean + precision_mean + recall_mean + f1_mean + roc_auc_mean + mcc_mean)

### Paso 8: Realizar la optimización de hiperparámetros
Se realiza la optimización de hiperparámetros utilizando Optuna con un número específico de ensayos.


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

### Paso 9: Obtener los mejores hiperparámetros y entrenar el modelo
Se obtienen los mejores hiperparámetros encontrados por Optuna y se entrena el modelo Random Forest con estos parámetros.


In [ ]:
best_params = study.best_params
rf = RandomForestClassifier(random_state=200, **best_params)
rf.fit(X_resampled, y_resampled)

### Paso 10: Guardar el modelo entrenado en un archivo pickle
El modelo entrenado se guarda en un archivo pickle para su uso futuro.



In [ ]:
with open('modelo_entrenadoSIN.pickle', 'wb') as f:
    pickle.dump(rf, f)

### Paso 11: Cargar el modelo entrenado y realizar predicciones en datos de validación
Se carga el modelo entrenado desde el archivo pickle y se realizan predicciones en el conjunto de datos de validación.


In [ ]:
with open('modelo_entrenadoSIN.pickle', 'rb') as f:
    loaded_rf_model = pickle.load(f)
y_pred_val_loaded = loaded_rf_model.predict(X_val)

### Paso 12: Evaluar el modelo en datos de validación
Se calculan métricas de evaluación como precisión, recuperación y se genera un informe de clasificación en el conjunto de datos de validación.

In [ ]:
precision_val_loaded = precision_score(y_val, y_pred_val_loaded)
recall_val_loaded = recall_score(y_val, y_pred_val_loaded)
print('Metricas de evaluacion en los datos de validacion usando el modelo cargado:')
print('Precision:', precision_val_loaded)
print('Recall:', recall_val_loaded)
print('Reporte de clasificacion en conjunto de datos de validacion:')
print(classification_report(y_val, y_pred_val_loaded))

### Paso 13: Visualizar la matriz de confusión
La matriz de confusión se visualiza en un mapa de calor utilizando la biblioteca Seaborn.


In [ ]:
cm_val_loaded = confusion_matrix(y_val, y_pred_val_loaded)
sns.heatmap(cm_val_loaded, annot=True, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

### Paso 14: Guardar la matriz de confusión en un archivo CSV
La matriz de confusión se guarda en un archivo CSV llamado 'matriz_validacion.csv' para su análisis posterior.
Siguiendo estos pasos, podrás entrenar un modelo de clasificación de moléculas, optimizar sus hiperparámetros y evaluar su rendimiento en un conjunto de datos de validación.


In [ ]:
df_cm_loaded = pd.DataFrame(cm_val_loaded, index=['Clase ' + str(i) for i in range(cm_val_loaded.shape[0])],
                            columns=['Prediccion ' + str(i) for i in range(cm_val_loaded.shape[1])])
df_cm_loaded.to_csv('matriz_validacionSIN.csv')

Script elaborado por Nelson Alejandro Amaya Orozco, como trabajo de grado, para el grupo de investigacion RamirezLAB